<a href="https://colab.research.google.com/github/YasmineSarraj/MLOPS-assignments/blob/main/Creation_of_tinyDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
#df = pd.read_csv("/Users/yasminesarraj/Documents/GitHub/MLOPS-assignments/Assignment_1/data/final_data.csv")
#We have used this dataset but can't upload it https://www.kaggle.com/datasets/datafiniti/hotel-reviews



FileNotFoundError: [Errno 2] No such file or directory: 'kaggle datasets download -d datafiniti/hotel-reviews'

In [5]:
len(df.Hotel_Address.unique())

1476

In [5]:
! pip install tinydb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from tinydb import TinyDB, Query 

In [7]:
hotel_reviews_db = TinyDB("hotel_reviews_db.json")

In [8]:
reviews = hotel_reviews_db.table("reviews")

In [10]:
from tqdm import tqdm

In [18]:
%%time

for _, row in df.iterrows():
    reviews.insert(row.to_dict())


KeyboardInterrupt: 

In [19]:
len(reviews.all())

33538

In [14]:
new_db = TinyDB("reviews_db.json")

In [15]:
revs = new_db.table("reviews")

In [16]:
names = []

for x in revs:
  name = x.get("Hotel_Name")
  if name not in names:
    names.append(name)

names

[]

Here we will load the sentiment analysis model

In [17]:
from transformers import pipeline

# Load the pre-trained sentiment analysis model
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Extract sentiment reviews for the movie reviews
reviews = reviews.all()

In [ ]:
for review in reviews:
  print(review)

In [ ]:
# Iterate over the movie reviews and update the sentiment label for each one
i = 0
for review in reviews:

    # Analyze the sentiment of the review using the pre-trained classifier
    sentiment = classifier(review['text'][:512])[0]['label']
    
    # Map the sentiment label to a binary label (1 for positive, 0 for negative)
    if sentiment == 'POSITIVE':
      label = 1
    else:
      label = 0
      
    # Update the 'sentiment' column in the 'reviews' table with the binary label for the current review
    reviews.update({'sentiment': label}, where('row_number') == i)
    i = i + 1